In [1]:
from script.scraper_functions import *
from script.soupifier import *
from serde.json import to_json
import numpy as np

## Le code ci-dessous lance le `scraper`

- Tu peux changer le nombre d'*URL* de **19** à 1 ou 2 pour tester ça prendra beaucoup moins longtemps à générer

In [2]:
session = create_session()
url_list = create_url_list(URL, 19)
pages = read_pages(url_list=url_list, session=session)

Page 1 -> Successfull Extraction.


Elapsed time for read_pages function: 0.97 seconds.

In [ ]:
all_urls = extract_all_urls(pages)
print("--------------------")
product_pages = extract_all_pages(all_urls, session)

Il faudrait créer une fonction pour la génération de la liste des smartphones + l'export en `json`

In [8]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [13]:
smartphones_json = to_json(smartphone_list)

In [14]:
file_path = "data/smartphones.json"

with open(file_path, "w", encoding="utf-8") as json_file:
    json_file.write(smartphones_json)

## Charge le Dataframe dans `polars`

In [3]:
import polars as pl
from script.polars_gazprom import *

In [4]:
df = pl.read_json("data/smartphones.json")

### Pipeline

In [5]:
df = NordStream(df)

Vérification des valeurs nulles sur les différentes colonnes.

In [6]:
df.select(pl.all().is_null().sum())

made_in,upgrade_storage,storage,net_weight,stars,screen_size,thickness,screen_tech,das_head,cpu,color,charging_type,screen_resolution,sensor,repairability_index,brand,das_chest,os,bluetooth,fast_charging,model,price,sensor_resolution,network,width,sim_number,screen_type,ram,das_limbs,sim_type,height,battery,reviews,usb_type_c
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
10,0,0,15,0,0,1,0,0,1,1,3,0,80,0,0,0,1,0,0,0,0,0,0,1,5,0,0,0,0,1,0,0,0


In [39]:
df.select(pl.col("model").filter(pl.col("net_weight").is_null())).unique().to_numpy()

array([['Sony Xperia 1 V'],
       ['Google Pixel 8'],
       ['Crosscall Stellar-X5'],
       ['Google Pixel 8 Pro'],
       ['Samsung Galaxy S10']], dtype=object)

Idée => Avec cette liste unique des téléphones, aller chercher leur poids (manquant) avec une `request` sur google et récupérer le résultat

In [7]:
df.filter(pl.col("charging_type").is_null())
# ces 3 téléphones sont en fait chargeables par induction

made_in,upgrade_storage,storage,net_weight,stars,screen_size,thickness,screen_tech,das_head,cpu,color,charging_type,screen_resolution,sensor,repairability_index,brand,das_chest,os,bluetooth,fast_charging,model,price,sensor_resolution,network,width,sim_number,screen_type,ram,das_limbs,sim_type,height,battery,reviews,usb_type_c
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,i64,bool
"""Viêt Nam""","""Micro SD""","""128 Go""","""197,00 g""",4.3137,"""6,6"" soit 16,8…","""8,40 mm""","""TFT (LCD)""","""1,492 W/kg""","""8 coeurs jusqu…","""Bleu""",null,"""2408 x 1080 pi…","""3""","""8,1 /10""","""SAMSUNG""","""1,251 W/kg""","""Android 12""","""5.1""","""Oui""","""Samsung Galaxy…",279.0,"""50 mégapixels+…","""5G""","""76,90 mm""","""2 SIM""","""Plat""","""4 Go""","""2,840 W/kg""","""Dual SIM""","""165,40 mm""","""5000 mAh""",51,true
"""Viêt Nam""","""Micro SD""","""128 Go""","""197,00 g""",4.3137,"""6,6"" soit 16,8…","""8,40 mm""","""TFT (LCD)""","""1,492 W/kg""","""8 coeurs jusqu…","""Noir""",null,"""2408 x 1080 pi…","""3""","""8,1 /10""","""SAMSUNG""","""1,251 W/kg""","""Android 12""","""5.1""","""Oui""","""Samsung Galaxy…",279.0,"""50 mégapixels+…","""5G""","""76,90 mm""","""2 SIM""","""Plat""","""4 Go""","""2,840 W/kg""","""Dual SIM""","""165,40 mm""","""5000 mAh""",51,true
"""Chine""","""Micro SD""","""64 Go""","""190,00 g""",4.6667,"""6,6"" soit 16,8…","""8,40 mm""","""LCD (LTPS) FHD…","""0,788 W/kg""","""8 coeurs jusqu…","""Noir""",null,"""1080 x 2400 pi…","""3""","""6,1 /10""","""REALME""","""1,020 W/kg""","""Android 11""","""5""","""Oui""","""Realme 9i""",219.0,"""50 mégapixels+…","""4G+""","""75,50 mm""","""1 SIM""","""Plat""","""4 Go""","""2,072 W/kg""","""Nano SIM""","""164,40 mm""","""5000 mAh""",3,true


In [8]:
df.filter(pl.col("made_in").is_null())
# on doit fill ces 10 pays là, ce ne doit pas etre très compliqué

made_in,upgrade_storage,storage,net_weight,stars,screen_size,thickness,screen_tech,das_head,cpu,color,charging_type,screen_resolution,sensor,repairability_index,brand,das_chest,os,bluetooth,fast_charging,model,price,sensor_resolution,network,width,sim_number,screen_type,ram,das_limbs,sim_type,height,battery,reviews,usb_type_c
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,i64,bool
null,"""Micro SD""","""16 Go""","""165,00 g""",3.8929,"""5,4"" soit 13,7…","""9,20 mm""","""IPS (LCD)""","""0,520 W/kg""","""4 coeurs jusqu…","""Gris""","""Pas de charge …","""1440 x 720 pix…","""1""","""6,0 /10""","""DORO""","""1,250 W/kg""","""Android 9.0 Pi…","""Oui""","""Non""","""DORO 8050""",230.3,"""13 mégapixels""","""4G""","""70,80 mm""","""1 SIM""","""Plat""","""2 Go""","""2,630 W/kg""","""Nano SIM et Mi…","""152,90 mm""","""3000 mAh""",28,true
null,"""NM Card""","""128 Go""","""183,00 g""",3.4054,"""6,4"" soit 16,3…","""8,70 mm""","""IPS (LCD)""","""0,850 W/kg""","""8 coeurs jusqu…","""Vert""","""Pas de charge …","""2340 x 1080 pi…","""4""","""6,7 /10""","""HUAWEI""","""1,150 W/kg""","""Android 10""","""5.1""","""Non""","""HUAWEI P40 Lit…",305.97,"""48 mégapixels+…","""4G""","""76,30 mm""","""2 SIM ou 1 SIM…","""Plat""","""6 Go""","""2,770 W/kg""","""Nano SIM x2 ou…","""159,20 mm""","""4200 mAh""",74,true
null,"""Non""","""256 Go""","""183,00 g""",5.0,"""6,7"" soit 17 c…","""6,90 mm""","""Dynamic AMOLED…","""0,799 W/kg""","""8 coeurs jusqu…","""Gris""","""Possibilité de…","""2636 x 1080 pi…","""2""","""5,1 /10""","""SAMSUNG""","""1,451 W/kg""","""Android 10""","""5""","""Oui""","""Samsung Galaxy…",1992.93,"""12 mégapixels+…","""5G""","""73,60 mm""","""2 SIM""","""Pliable""","""8 Go""","""1,790 W/kg""","""Nano SIM et eS…","""167,30 mm""","""3300 mAh""",2,true
null,"""Micro SD""","""128 Go""","""203,00 g""",4.2857,"""6,7"" soit 17 c…","""8,40 mm""","""Super AMOLED (…","""0,230 W/kg""","""8 coeurs jusqu…",null,"""Pas de charge …","""2400 x 1080 pi…","""4""","""8,2 /10""","""SAMSUNG""","""1,168 W/kg""","""Android 11""","""5""","""Oui""","""Samsung Galaxy…",483.9,"""64 mégapixels+…","""4G""","""77,40 mm""","""2 SIM""","""Plat""","""6 Go""","""2,490 W/kg""","""Nano SIM (x2)""","""165,00 mm""","""5000 mAh""",14,true
null,"""SD""","""64 Go""","""199,00 g""",4.0161,"""6,5"" soit 16,5…","""8,90 mm""","""IPS (LCD)""","""0,795 W/kg""","""8 coeurs jusqu…","""Vert""","""Pas de charge …","""2400 x 1080 pi…","""4""","""7,1 /10""","""XIAOMI""","""1,138 W/kg""","""Android 9.0 Pi…","""5""","""Oui""","""XIAOMI Redmi N…",171.04,"""48 mégapixels+…","""4G""","""77,20 mm""","""2 SIM""","""Plat""","""3 Go""","""2,262 W/kg""","""SIM""","""162,30 mm""","""5020 mAh""",62,true
null,"""Non""","""256 Go""","""202,00 g""",4.75,"""6,7"" soit 17 c…","""7,80 mm""","""Dynamic AMOLED…","""0,541 W/kg""",null,"""Gris""","""Possibilité de…","""2400 x 1080 pi…","""3""","""8,2 /10""","""SAMSUNG""","""1,335 W/kg""","""Android 11""","""5.2""","""Oui""","""Samsung Galaxy…",1274.99,"""64 mégapixels+…","""5G""","""75,80 mm""","""2 SIM""","""Plat""","""8 Go""","""2,230 W/kg""","""SIM""","""161,40 mm""","""4800 mAh""",4,true
null,"""Non""","""128 Go""","""184,00 g""",4.4568,"""6,6"" soit 16,8…","""8,10 mm""","""LTPS""","""0,880 W/kg""","""8 coeurs jusqu…","""Noir""","""Pas de charge …","""2400 x 1080 pi…","""4""","""7,3 /10""","""OPPO""","""0,905 W/kg""","""Android 10""","""5""","""Oui""","""OPPO Reno 4Z""",288.49,"""48 mégapixels+…","""5G""","""75,50 mm""","""2 SIM""","""Bord à bord""","""8 Go""","""2,686 W/kg""","""Nano SIM (x2)""","""163,80 mm""","""4000 mAh""",81,true
null,"""Micro SD""","""128 Go""","""193,00 g""",4.5217,"""6,5"" soit 16,5…","""8,40 mm""","""Super AMOLED (…","""0,503 W/kg""","""Qualcomm Snapd…","""Vert""","""Possibilité de…","""2400 x 1080 pi…","""3""","""8,1 /10""","""SAMSUNG""","""1,335 W/kg""","""Android 10""","""5""","""Oui""","""Samsung Galaxy…",391.9,"""12 mégapixels+…","""5G""","